In [15]:
from datasets import load_dataset, DatasetDict
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, pipeline
import torch
import pandas as pd
import numpy as np
import random
import re

In [4]:
df = pd.read_csv("data/ultra-dm-dataset.csv")

In [7]:
#for the dm-model, we follow a system_prompt|user_prompt|assistant_response	format for finetuning, so we make a specialized dataloader
class TinyLlamaDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        messages = [
            {"role": "system", "content": self.data.iloc[idx]["system_prompt"]},
            {"role": "user", "content": self.data.iloc[idx]["user_prompt"]},
            {"role": "assistant", "content": self.data.iloc[idx]["assistant_response"]},
        ]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        encoded = tokenizer(prompt, truncation=True, padding="max_length", return_tensors="pt")
        input_ids = encoded["input_ids"].squeeze()
        attention_mask = encoded["attention_mask"].squeeze()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids.clone(),
        }

dataset = TinyLlamaDataset(df)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    cache_dir="/projectnb/cs505aw/students/samwu/risk-llm/")

In [11]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

#model training args
training_args = TrainingArguments(
    output_dir="dm-model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=torch.cuda.is_available(),  
    report_to="none"
)

#trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/scratch/4280996.1.l40s/ipykernel_3692555/3362493372.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=42, training_loss=0.8535274323962984, metrics={'train_runtime': 85.4112, 'train_samples_per_second': 1.967, 'train_steps_per_second': 0.492, 'total_flos': 2135630813331456.0, 'train_loss': 0.8535274323962984, 'epoch': 3.0})

In [13]:
scenarios = pd.read_csv("data/scenario-dataset.csv")["scenarios"].to_list()

In [34]:
attributes = np.random.rand(7)
text = random.choice(scenarios)
dm_match = re.search(r"<DM>(.*?)</DM>", text)
player_match = re.search(r"<Player>(.*?)</Player>", text)
dm_text = dm_match.group(1) if dm_match else ""
player_text = player_match.group(1) if player_match else ""

roll = "Success" if random.random() < 0.5 else "Failure"
output = f"(health:{attributes[0]:.2f}\nstrength:{attributes[1]:.2f}\ndexterity:{attributes[2]:.2f}\nperception:{attributes[3]:.2f}\nintelligence:{attributes[4]:.2f}\ncharisma:{attributes[5]:.2f}\nstamina:{attributes[6]:.2f})\n<DM>{dm_text}</DM>\n<Roll:{roll}>"

#feed output through tokenizer
#define the chat messages
messages = [
    {
        "role": "system",
        "content": text,
    },
    {"role": "user", "content": player_text},
]

#use the tokenizer's chat template to construct the prompt
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# generate output
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        max_new_tokens=97,
        do_sample=True,
        top_p=0.95,
        temperature=1.0,
    )
# decode and print the output
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
response = generated_text.split("<|assistant|>")[-1]
print(f"System:{output}")
print(f"User:{player_text}")
print("____")
print(f"Response:{response}")

System:(health:0.34
strength:0.94
dexterity:0.32
perception:0.52
intelligence:0.70
charisma:0.36
stamina:0.97)
<DM>You’re walking through a fog-covered forest. The trees loom overhead, their branches twisted and gnarled. </DM>
<Roll:Failure>
User:I try to stay alert, keeping an eye out for any hidden creatures or dangers lurking in the fog.
____
Response:
As you walk cautiously through the fog-covered forest, your attention wavers as a faint noise echoes from the shadows. The trees seem to move around you, and the unsettling atmosphere of the forest heightens your senses. You hear a low, rhythmic clicking from the bushes on either side of your path, followed by the sharp snap of a branch. Suddenly, a pair of glowing eyes stare back at you
